In [11]:
!pip install streamlit transformers accelerate sentencepiece pyngrok -q


In [12]:
!pip install streamlit ctransformers==0.2.27 langchain-core pyngrok huggingface_hub -q


In [28]:
%%writefile app.py
import os
import streamlit as st
from ctransformers import AutoModelForCausalLM
from langchain_core.prompts import PromptTemplate

# Streamlit config
st.set_page_config(page_title="Generate Emails", page_icon="📧", layout="centered")
st.header("Generate Emails 📧")

# Sidebar model settings
with st.sidebar:
    st.subheader("Model Settings")
    gpu_layers = st.number_input("GPU layers (0 = CPU only)", min_value=0, max_value=100, value=0, step=1)
    max_new_tokens = st.slider("Max new tokens", 64, 1024, 256, 32)
    temperature = st.slider("Temperature", 0.0, 1.5, 0.7, 0.05)

@st.cache_resource(show_spinner=True)
def load_llm(gpu_layers):
    return AutoModelForCausalLM.from_pretrained(
        "TheBloke/Llama-2-7B-Chat-GGML",             # repo name
        model_file="llama-2-7b-chat.ggmlv3.q4_0.bin", # file in repo
        model_type="llama",
        gpu_layers=int(gpu_layers),
        context_length=4096,
        threads=os.cpu_count(),
    )

llm = load_llm(gpu_layers)

SYSTEM_PROMPT = (
    "You are an expert assistant that writes clear, concise, and polite emails. "
    "Keep the email well-structured, professional, and tailored to the requested style."
)

TEMPLATE = """<s>[INST] <<SYS>>
{system_prompt}
<</SYS>>

Write an email with {style} style that includes the topic: {email_topic}.

Sender: {sender}
Recipient: {recipient}

Constraints:
- Start with a greeting (e.g., Dear {recipient}).
- Keep it concise and specific to the topic.
- Close with a proper sign-off using the sender's name.
[/INST]
"""

prompt_tmpl = PromptTemplate(
    input_variables=["system_prompt", "style", "email_topic", "sender", "recipient"],
    template=TEMPLATE,
)

def getLLMResponse(form_input, email_sender, email_recipient, email_style):
    prompt = prompt_tmpl.format(
        system_prompt=SYSTEM_PROMPT,
        email_topic=form_input.strip(),
        sender=email_sender.strip(),
        recipient=email_recipient.strip(),
        style=email_style.strip(),
    )
    text = llm(
        prompt,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        stop=["</s>"]
    )
    return text.strip()

# UI form
form_input = st.text_area('Enter the email topic', height=200)
col1, col2, col3 = st.columns([10, 10, 5])
with col1:
    email_sender = st.text_input('Sender Name')
with col2:
    email_recipient = st.text_input('Recipient Name')
with col3:
    email_style = st.selectbox('Writing Style', ('Formal', 'Appreciating', 'Not Satisfied', 'Neutral'))

disabled = not (form_input and email_sender and email_recipient)

if st.button("Generate", disabled=disabled):
    st.subheader("Generated Email")
    email_text = getLLMResponse(form_input, email_sender, email_recipient, email_style)
    st.write(email_text)


Overwriting app.py


In [26]:
!pkill ngrok


In [29]:
from pyngrok import ngrok
import threading
import subprocess

# Set your ngrok token
from google.colab import userdata

# Retrieve ngrok token securely
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start tunnel
public_url = ngrok.connect(8501)
print(f"Access the app here: {public_url}")

# Run the app
def run_app():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"])

thread = threading.Thread(target=run_app)
thread.start()

Access the app here: NgrokTunnel: "https://a344ed1e17dc.ngrok-free.app" -> "http://localhost:8501"
